In [1]:
import os
import sys

def is_colab():
    return 'google.colab' in sys.modules

if is_colab():
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_PATH = '/content/drive/MyDrive/NBS_Project'
else:
    # 自动根据当前脚本定位项目根路径
    PROJECT_PATH = os.path.abspath(os.path.join(os.path.dirname(__file__), '..')) \
        if '__file__' in globals() else os.getcwd()

if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)

print(f"项目路径设置为: {PROJECT_PATH}")

项目路径设置为: c:\Users\Barytes\Desktop\NBS_project


In [2]:
from src.config import Config
from src.main import create_MDs,create_ESP
import src.baseline as baseline
import src.NBS as NBS
import src.model as model
import numpy as np

CONFIG_PATH = os.path.join(PROJECT_PATH, "configs", "test.json")
config = Config(CONFIG_PATH)

# Create MDs with a specific seed for reproducibility
seed = 41
MDs = create_MDs(config, seed=seed)
# np.random.shuffle(MDs)  # Shuffle the MDs

for md in MDs:
    print(md.param)

esp = create_ESP(config, seed=seed)
print(esp.param)

{'s': 0.1, 'l': 0.8, 'cn': 0.32332202513299674, 'Fn': 1.184383282690447, 'kn': 0.6732799162883472, 'omega_n': 1, 'Rn': 4.13472934445215}
{'s': 0.1, 'l': 0.8, 'cn': 0.6374404631644437, 'Fn': 1.7637226511189565, 'kn': 0.6651454029151388, 'omega_n': 1, 'Rn': 10.248845012378245}
{'s': 0.1, 'l': 0.8, 'cn': 0.395711266542491, 'Fn': 2.1321345406869066, 'kn': 0.19255662205025992, 'omega_n': 1, 'Rn': 6.716553285779903}
{'s': 0.1, 'l': 0.8, 'cn': 0.16187321635232887, 'Fn': 3.8199302863134226, 'kn': 0.3183833952999494, 'omega_n': 1, 'Rn': 9.231363952354403}
{'s': 0.1, 'l': 0.8, 'cn': 0.6413215489755134, 'Fn': 3.913825949541608, 'kn': 0.4233228768151258, 'omega_n': 1, 'Rn': 7.948101841125762}
{'s': 0.1, 'l': 0.8, 'cn': 0.49308149762923636, 'Fn': 2.4920855793255217, 'kn': 0.5819339189335254, 'omega_n': 1, 'Rn': 3.859406010253179}
{'s': 0.1, 'l': 0.8, 'cn': 0.17404615893711323, 'Fn': 1.5048957788256199, 'kn': 0.3264336993988291, 'omega_n': 1, 'Rn': 7.130152926250668}
{'s': 0.1, 'l': 0.8, 'cn': 0.350

In [3]:
T = 100
N = len(MDs)
lam_uni, p_uni, r_uni, Dmax_uni = np.zeros(N),np.zeros(N),np.zeros(N),0
for _ in range(T):
    l,p,r,d = baseline.uniform_baseline(esp, MDs)
    lam_uni += l
    p_uni += p 
    r_uni += r
    Dmax_uni += d
lam_uni, p_uni, r_uni, Dmax_uni = lam_uni/T, p_uni/T, r_uni/T, Dmax_uni/T

print(f"Uniform λ: {lam_uni}")
print(f"Uniform p: {p_uni}")  
print(f"Uniform Dmax: {Dmax_uni}")
print(f"Uniform r: {r_uni}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_uni, p_uni))
print("sum lambda:", np.sum(lam_uni), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_uni[i],lam_uni[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_uni, " D0:", esp.D0)
max_Dn = max(md.delay(p_uni[i], lam_uni[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_uni == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_uni[i], lam_uni[i]) <= Dmax_uni for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_uni <= esp.D0)
print("pn<=Fn:", all(p_uni[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_uni[i] <= p_uni[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_uni), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_uni[i] >= md.Ln(p_uni[i]) for i, md in enumerate(MDs)))

AttributeError: module 'numpy.random' has no attribute 'default_rng'

In [3]:
lam_pro, p_pro, r_pro, Dmax_pro = baseline.proportional_baseline(esp,MDs)
print(f"Uniform λ: {lam_pro}")
print(f"Uniform p: {p_pro}")  
print(f"Uniform Dmax: {Dmax_pro}")
print(f"Uniform r: {r_pro}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_pro, p_pro))
print("sum lambda:", np.sum(lam_pro), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_pro[i],lam_pro[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_pro, " D0:", esp.D0)
max_Dn = max(md.delay(p_pro[i], lam_pro[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_pro == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_pro[i], lam_pro[i]) <= Dmax_pro for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_pro <= esp.D0)
print("pn<=Fn:", all(p_pro[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_pro[i] <= p_pro[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_pro), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_pro[i] >= md.Ln(p_pro[i]) for i, md in enumerate(MDs)))

Uniform λ: [2.09025991 3.11270752 3.76289958 6.74160743 6.90731927 4.39815949
 2.65591668 5.8750097  6.85821515 7.59790528]
Uniform p: [1.18438328 1.76372265 2.13213454 3.81993029 3.91382595 2.49208558
 1.50489578 3.32889859 3.88600257 4.30512586]
Uniform Dmax: 0.1028355471996345
Uniform r: [ 8.10768535 12.07354791 14.59550826 26.14929921 26.79206114 17.05954993
 10.30175083 22.78794606 26.60159642 29.47070127]
social welfare: 193.93964638257935
sum lambda: 50.0  lambda0: 50
['MD 1 Dn: 0.1028355471996345', 'MD 2 Dn: 0.0625727251662071', 'MD 3 Dn: 0.05857811286095347', 'MD 4 Dn: 0.03521840591599758', 'MD 5 Dn: 0.03638235768232111', 'MD 6 Dn: 0.06328983391342033', 'MD 7 Dn: 0.07592421010504175', 'MD 8 Dn: 0.03795766576644862', 'MD 9 Dn: 0.03604407740719918', 'MD 10 Dn: 0.03910697341732085']
Dmax: 0.1028355471996345  D0: 3
max Dn: 0.1028355471996345
Dmax == max Dn: True
Dn<=Dmax: True
Dmax<=D0: True
pn<=Fn: True
lambda_n<=pn/sl: True
sum r: 193.93964638257935 Q_star: 248.274174600214
rn>=

In [4]:
lam_non, p_non, r_non, Dmax_non = baseline.non_cooperative_baseline(esp,MDs)
print(f"Non-cooperative λ: {lam_non}")
print(f"Non-cooperative p: {p_non}")  
print(f"Non-cooperative Dmax: {Dmax_non}")
print(f"Non-cooperative r: {r_non}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_non, p_non))
print("sum lambda:", np.sum(lam_non), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_non[i],lam_non[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_non, " D0:", esp.D0)
max_Dn = max(md.delay(p_non[i], lam_non[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_non == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_non[i], lam_non[i]) <= Dmax_non for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_non <= esp.D0)
print("pn<=Fn:", all(p_non[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_non[i] <= p_non[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_non), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_non[i] >= md.Ln(p_non[i]) for i, md in enumerate(MDs)))

d:\Program\Anaconda\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:187: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  'approximations.', UserWarning)
c:\Users\Barytes\Desktop\NBS_project\src\model.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  Dmax = max([md.s/md.Rn + 1/(pn/(md.s*md.l)-lamn) for (md,pn,lamn) in zip(MDs,p,lam)])


✅ 这个点严格满足所有约束（在容差范围内）。
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -245.40286262770385
            Iterations: 61
            Function evaluations: 1404
            Gradient evaluations: 61
Non-cooperative λ: [0.00000000e+00 2.56406507e-01 7.79001398e+00 1.93402040e+01
 3.73601309e+00 2.50934455e+00 9.19819000e+00 2.63651560e-15
 7.16982791e+00 5.94640676e-15]
Non-cooperative p: [0, 0.12454902780519324, 0.7279365410320255, 1.6513985287071118, 0.4033010934453515, 0.30701645716982323, 0.8404723347792799, 0, 0.6779369936081128, 0]
Non-cooperative Dmax: 0.7787180367491368
Non-cooperative r: [0.07220914 0.07923238 0.29908612 0.69418338 0.1844806  0.17178413
 0.39061184 0.08050852 0.295071   0.07901784]


TypeError: unsupported operand type(s) for -: 'NoneType' and 'float'

In [7]:
print([lam_non[i] <= p_non[i] / (md.s*md.l) for i, md in enumerate(MDs)])

[False, True, True, True, True, True, True, False, True, False]


In [5]:
Dmax = max([md.s/md.Rn + 1/(pn/(md.s*md.l)-lamn) for (md,pn,lamn) in zip(MDs,p_non,lam_non)])
print(Dmax,esp.D0)
print(esp.D0 < Dmax-1e-3)

inf 3
True


d:\Program\Anaconda\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in double_scalars
  """Entry point for launching an IPython kernel.


In [5]:
lam_swm, p_swm, Dmax_swm = baseline.social_welfare_maximization(esp, MDs)
print(f"SWM λ: {lam_swm}")
print(f"SWM p: {p_swm}")  
print(f"SWM Dmax: {Dmax_swm}")
r_swm = baseline.solve_r_NBP(esp, MDs, Dmax_swm, lam_swm, p_swm)
print(f"SWM r: {r_swm}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_swm, p_swm))
print("sum lambda:", np.sum(lam_swm), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_swm[i],lam_swm[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_swm, " D0:", esp.D0)
max_Dn = max(md.delay(p_swm[i], lam_swm[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_swm == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_swm[i], lam_swm[i]) <= Dmax_swm for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_swm <= esp.D0)
print("pn<=Fn:", all(p_swm[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_swm[i] <= p_swm[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_swm), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_swm[i] >= md.Ln(p_swm[i]) for i, md in enumerate(MDs)))

d:\Program\Anaconda\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:187: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  'approximations.', UserWarning)


✅ 这个点严格满足所有约束（在容差范围内）。
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -243.40324662776618
            Iterations: 127
            Function evaluations: 2980
            Gradient evaluations: 127
SWM λ: [2.7936761  2.31368116 6.66632346 9.5806704  4.92043884 4.2690617
 7.17361526 2.37876972 6.75656014 3.14720321]
SWM p: [0.32709068 0.286791   0.63567011 0.86828936 0.49569806 0.44535351
 0.67614046 0.29202623 0.64252157 0.35447966]
SWM Dmax: 0.7964125679109589
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -37.58977177842789
            Iterations: 12
            Function evaluations: 145
            Gradient evaluations: 12
SWM r: [20.68276245 20.64382986 20.66515329 20.67182551 20.68587363 20.7109738
 20.71078741 20.67500694 20.69903351 20.72764555]
social welfare: 245.14823407853794
sum lambda: 50.0  lambda0: 50
['MD 1 Dn: 0.7964115679548526', 'MD 2 Dn: 0.7964115679483934', 'MD 3 Dn: 0.796411567939

In [6]:
lam_opt, p_opt, r_opt, Dmax_opt = baseline.optimal_NBP(esp, MDs)
print(f"Optimal λ: {lam_opt}")
print(f"Optimal p: {p_opt}")  
print(f"Optimal r: {r_opt}")
print(f"Optimal Dmax: {Dmax_opt}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_opt, p_opt))
print("sum lambda:", np.sum(lam_opt), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_opt[i],lam_opt[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_opt, " D0:", esp.D0)
print("max Dn:", max(md.delay(p_opt[i], lam_opt[i]) for i, md in enumerate(MDs)))
print("Dmax == max Dn:", Dmax_opt == max(md.delay(p_opt[i], lam_opt[i]) for i, md in enumerate(MDs)))
print("Dn<=Dmax:", all(md.delay(p_opt[i], lam_opt[i]) <= Dmax_opt for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_opt <= esp.D0)
print("pn<=Fn:", all(p_opt[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_opt[i] <= p_opt[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_opt), "Q_star:", esp.Q(Dmax_opt))
print("rn>=Ln:", all(r_opt[i] >= md.Ln(p_opt[i]) for i, md in enumerate(MDs)))

d:\Program\Anaconda\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:187: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  'approximations.', UserWarning)


✅ 这个点严格满足所有约束（在容差范围内）。
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -35.98812131832307
            Iterations: 59
            Function evaluations: 1948
            Gradient evaluations: 59
Optimal λ: [4.84048218 4.74769353 5.1733058  5.31423328 4.98065634 4.93264803
 5.19159526 4.82942949 5.14613748 4.84381861]
Optimal p: [0.48926739 0.48000077 0.51469772 0.52545906 0.49899342 0.49686566
 0.51605123 0.48656696 0.51216767 0.48866783]
Optimal r: [20.8111587  20.82055493 20.56067607 20.51122049 20.65549879 20.72429146
 20.58863828 20.84045136 20.58289172 20.83150454]
Optimal Dmax: 0.808277617873896
social welfare: 244.7511247091813
sum lambda: 50.0  lambda0: 50
['MD 1 Dn: 0.8082776179066538', 'MD 2 Dn: 0.8082776178374794', 'MD 3 Dn: 0.8082776178869839', 'MD 4 Dn: 0.8082776178591748', 'MD 5 Dn: 0.8082776178740338', 'MD 6 Dn: 0.8082776178759148', 'MD 7 Dn: 0.8082776178769785', 'MD 8 Dn: 0.808277617882433', 'MD 9 Dn: 0.8082776178881402', 'MD 10 

In [7]:
lamb_admm, p_admm, Dmax_admm = NBS.ADMM(esp, MDs)
print(f"ADMM λ: {lamb_admm}")
print(f"ADMM p: {p_admm}")  
print(f"ADMM Dmax: {Dmax_admm}")
r_admm = NBS.negotiation(esp, MDs,lamb_admm, p_admm, Dmax_admm)
print(f"ADMM r: {r_admm}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lamb_admm, p_admm))
print("sum lambda:", np.sum(lamb_admm), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_admm[i],lamb_admm[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_admm, " D0:", esp.D0)
print("max Dn:", max(md.delay(p_admm[i], lamb_admm[i]) for i, md in enumerate(MDs)))
print("Dmax == max Dn:", Dmax_admm == max(md.delay(p_admm[i], lamb_admm[i]) for i, md in enumerate(MDs)))
print("Dn<=Dmax:", all(md.delay(p_admm[i], lamb_admm[i]) <= Dmax_admm for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_admm <= esp.D0)
print("pn<=Fn:", all(p_admm[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lamb_admm[i] <= p_admm[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_admm), "Q_star:", esp.Q(Dmax_admm))
print("rn>=Ln:", all(r_admm[i] >= md.Ln(p_admm[i]) for i, md in enumerate(MDs)))

MD SLSQP failed: Iteration limit exceeded
ADMM λ: [3.7425965  3.28905239 6.23274678 7.75015316 4.79432653 4.43719053
 6.49842393 3.55726373 6.02970723 3.6685392 ]
ADMM p: [0.40231988 0.36416106 0.60031556 0.72118595 0.48494466 0.45811631
 0.62145815 0.38564586 0.5837097  0.39551399]
ADMM Dmax: 0.8015498395542375
ADMM r: [20.74322653 20.71016821 20.63165597 20.60061758 20.66553611 20.71083742
 20.6672791  20.75575847 20.64844938 20.7567785 ]
social welfare: 245.01216618098908
sum lambda: 50.000000000000014  lambda0: 50
['MD 1 Dn: 0.8015473607214391', 'MD 2 Dn: 0.801547381740688', 'MD 3 Dn: 0.8015482922281147', 'MD 4 Dn: 0.8015520074256207', 'MD 5 Dn: 0.8015476651131107', 'MD 6 Dn: 0.8015474627751893', 'MD 7 Dn: 0.8015486954411308', 'MD 8 Dn: 0.8015464668038227', 'MD 9 Dn: 0.8015482695804274', 'MD 10 Dn: 0.8015461019662413']
Dmax: 0.8015498395542375  D0: 3
max Dn: 0.8015520074256207
Dmax == max Dn: False
Dn<=Dmax: False
Dmax<=D0: True
pn<=Fn: True
lambda_n<=pn/sl: True
sum r: 206.8903072

In [8]:
print("Nash Product Opt:",model.nash_product_log(esp, MDs, lam_opt, p_opt, r_opt))
print("Nash Product SWM:",model.nash_product_log(esp, MDs, lam_swm, p_swm, r_swm))
print("Nash Product ADMM:",model.nash_product_log(esp, MDs, lamb_admm, p_admm, r_admm))
print("Nash Product Proportional:",model.nash_product_log(esp, MDs, lam_pro, p_pro, r_pro))
print("Nash Product Uniform:",model.nash_product_log(esp, MDs, lam_uni, p_uni, r_uni))
print("=============================")
print("SW Opt:",model.social_welfare(esp, MDs, lam_opt, p_opt))
print("SW SWM:",model.social_welfare(esp, MDs, lam_swm, p_swm))
print("SW ADMM:",model.social_welfare(esp, MDs, lamb_admm, p_admm))
print("SW Proportional:",model.social_welfare(esp, MDs, lam_pro, p_pro))
print("SW Uniform:",model.social_welfare(esp, MDs, lam_uni, p_uni))
print("=============================")
print("Dmax Opt:", Dmax_opt)
print("Dmax SWM:", Dmax_swm)
print("Dmax ADMM:", Dmax_admm)
print("Dmax Proportional:", Dmax_pro)
print("Dmax Uniform:", Dmax_uni)

Nash Product Opt: 37.5703170844825
Nash Product SWM: 37.58977133932888
Nash Product ADMM: 37.58310896998472
Nash Product Proportional: 33.65776480543405
Nash Product Uniform: 36.19223869212411
SW Opt: 244.7511247091813
SW SWM: 245.14823407853794
SW ADMM: 245.01216618098908
SW Proportional: 193.93964638257935
SW Uniform: 242.08589301080127
Dmax Opt: 0.808277617873896
Dmax SWM: 0.7964125679109589
Dmax ADMM: 0.8015498395542375
Dmax Proportional: 0.1028355471996345
Dmax Uniform: 1.4162059331166177
